In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import os
from sklearn import datasets, cross_validation
from classifier import KernelNullSpaceClassifier

In [2]:
PATH = r"/home/yen/Projects/masters/Datasets/Sample/"

In [3]:
ds = datasets.load_files(PATH)
ds.data = np.vstack([np.fromstring(txt, sep='\t') for txt in ds.data])

In [4]:
# Train novelty classifier
# Randomly select a few classes to represent the "novel" class.
data = ds.data
target = ds.target

classes = np.unique(target)
num_class = len(classes)
num_novel = max(np.floor(num_class*0.2), 1) # 20% of classes will be "novel"
novel = np.random.choice(target, num_novel)

X_train, X_test, y_train, y_test = cross_validation.train_test_split(\
    data, target, test_size=0.4, random_state=0)

# Remove all instances of the novel classes from the training set
mask = np.array([y not in novel for y in y_train])
X_train = X_train[mask]
y_train = y_train[mask]

# Test labels are 1 if novel, otherwise 0.
y_test = np.array([1 if cl in novel else 0 for cl in y_test])

In [5]:
classifier = KernelNullSpaceClassifier(threshold=50).fit(X_train, y_train)
classifier.predict(X_test)

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1])